# Voorbeeld van een agent voor het boeken van hotels en vluchten

Deze oplossing helpt je bij het boeken van vliegtickets en hotels. Het scenario is een reis van Londen Heathrow LHR op 20 februari 2024 naar New York JFK, met terugkeer op 27 februari 2025, vliegend in economy class uitsluitend met British Airways. Ik wil verblijven in een Hilton hotel in New York, graag de kosten voor de vlucht en het hotel opgeven.


# Initialiseer de Azure AI Agent Service en verkrijg configuratie-informatie uit **.env**

### **.env** 

Maak een .env-bestand aan 

**.env** bevat de verbindingsreeks van Azure AI Agent Service, het model dat door AOAI wordt gebruikt, en de bijbehorende Google API Search service API, ENDPOINT, enz.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Naam van jouw Azure AI Agent Service Model Deployment"

[**NOTE**] Je hebt een model nodig met een Rate Limit van 100.000 (Tokens per minuut) en een Rate Limit van 600 (Verzoeken per minuut).

  Je kunt een model verkrijgen in Azure AI Foundry - Model en Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Verbindingsreeks van jouw Azure AI Agent Service Project"

  Je kunt de projectverbindingsreeks vinden in het projectoverzicht op het AI ​​Foundry Portal-scherm.

- **SERPAPI_SEARCH_API_KEY** = "Jouw SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Jouw SERPAPI Search Endpoint"

Om de Model Deployment Name en Project Connection String van Azure AI Agent Service te verkrijgen, moet je Azure AI Agent Service aanmaken. Het wordt aanbevolen om [deze template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) te gebruiken om het direct aan te maken (***Let op:*** Azure AI Agent Service is momenteel ingesteld in een beperkt aantal regio's. Het wordt aanbevolen om [deze link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) te raadplegen om de regio in te stellen).

De agent moet toegang hebben tot SERPAPI. Het wordt aanbevolen om je te registreren via [deze link](https://serpapi.com/searches). Na registratie kun je een unieke API KEY en ENDPOINT verkrijgen.


# Installatie

Om dit notebook uit te voeren, moet je ervoor zorgen dat je de benodigde bibliotheken hebt geïnstalleerd door `pip install -r requirements.txt` uit te voeren.


In [ ]:
from semantic_kernel import __version__

__version__

Je Semantic Kernel-versie moet minimaal 1.27.2 zijn.


Laad uw .env-bestand instellingen en bronnen, zorg ervoor dat u uw sleutels en instellingen hebt toegevoegd en een lokaal .env-bestand hebt aangemaakt.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Inloggen bij Azure

Je moet nu inloggen bij Azure. Open een terminal en voer de volgende opdracht uit:

```bash
az login
```

Deze opdracht vraagt je om je Azure-inloggegevens in te voeren, zodat de Azure AI Agent-service correct kan functioneren.


# Uitleg:
Dit is een variabele die de API-sleutel opslaat voor toegang tot een SERP (Search Engine Results Page) API-service. Een API-sleutel is een unieke identificatie die wordt gebruikt om verzoeken te verifiëren die aan jouw account zijn gekoppeld.

Doel: Het doel van deze regel is om de API-sleutel in een variabele op te slaan, zodat deze kan worden gebruikt om verzoeken aan de SERP API-service te verifiëren. De API-sleutel is vereist om toegang te krijgen tot de service en zoekopdrachten uit te voeren.
Hoe een SERP API-sleutel te verkrijgen: Volg deze algemene stappen op https://serpapi.com om een SERP API-sleutel te verkrijgen (de exacte stappen kunnen variëren afhankelijk van de specifieke SERP API-service die je gebruikt):

Kies een SERP API-service: Er zijn verschillende SERP API-services beschikbaar, zoals SerpAPI, Google Custom Search JSON API en anderen. Kies degene die het beste bij jouw behoeften past.

Registreer een account: Ga naar de website van de gekozen SERP API-service en registreer een account. Je moet mogelijk wat basisinformatie verstrekken en je e-mailadres verifiëren.

Maak een API-sleutel aan: Nadat je je hebt geregistreerd, log in op je account en navigeer naar de API-sectie of het dashboard. Zoek naar een optie om een nieuwe API-sleutel aan te maken of te genereren.
Kopieer de API-sleutel naar je .env-bestand.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Uitleg:
BASE_URL: Dit is een variabele die de basis-URL opslaat voor het SERP API-eindpunt. De variabelenaam BASE_URL is een conventie die aangeeft dat deze URL het startpunt is voor het maken van API-verzoeken.  
'https://serpapi.com/search':  

Dit is de daadwerkelijke URL-string die aan de BASE_URL-variabele is toegewezen. Het vertegenwoordigt het eindpunt voor het uitvoeren van zoekopdrachten met behulp van de SERP API.

# Doel:
Het doel van deze regel is om een constante te definiëren die de basis-URL voor de SERP API bevat. Deze URL zal worden gebruikt als het startpunt voor het opbouwen van API-verzoeken om zoekoperaties uit te voeren.

# Gebruik:
Door de basis-URL in een variabele te definiëren, kun je deze eenvoudig hergebruiken in je code telkens wanneer je verzoeken moet doen aan de SERP API. Dit maakt je code beter onderhoudbaar en vermindert het risico op fouten door de URL op meerdere plaatsen hard te coderen. Het huidige voorbeeld is https://serpapi.com/search?engine=bing, dat gebruikmaakt van de Bing-zoek-API. Een andere API kan worden geselecteerd op https://Serpapi.com.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Uitleg:

Hier bevindt zich de code van je plugin.

Classdefinitie: `class BookingPlugin`: Definieert een klasse genaamd BookingPlugin die methoden bevat voor het boeken van hotels en vluchten.

Methode voor hotelreserveringen:

- `@kernel_function(description="booking hotel")`: Een decorator die de functie beschrijft als een kernel-functie voor het boeken van hotels.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definieert een methode voor het boeken van hotels met geannoteerde parameters en een geretourneerd type.

De methode stelt een woordenboek samen met parameters voor het hotelreserveringsverzoek en stuurt een GET-verzoek naar de SERP API. Het controleert de status van de respons en retourneert de hotelgegevens als het succesvol is, of None als het verzoek is mislukt.

Methode voor vluchtreserveringen:

- `@kernel_function(description="booking flight")`: Een decorator die de functie beschrijft als een kernel-functie voor het boeken van vluchten.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definieert een methode voor het boeken van vluchten met geannoteerde parameters en een geretourneerd type.

De methode stelt woordenboeken samen met parameters voor de uitgaande en terugkerende vluchtverzoeken en stuurt GET-verzoeken naar de SERP API. Het controleert de status van de respons en voegt de vluchtinformatie toe aan de resultaatstring als het succesvol is, of geeft een foutmelding als het verzoek is mislukt. De methode retourneert de resultaatstring met de vluchtinformatie.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Uitleg:
Importverklaringen: Importeer noodzakelijke modules voor Azure-referenties, AI-agent, chatberichtinhoud, auteursrol en kernelfunctie-decorator.

Asynchrone Contextmanager: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Dit stelt een asynchrone contextmanager in om Azure-referenties te beheren en een AI-agentclient te maken.

Agentnaam en Instructies: 
- `AGENT_NAME = "BookingAgent"`: Definieert de naam van de agent.
- `AGENT_INSTRUCTIONS = """..."""`: Biedt gedetailleerde instructies voor de agent over hoe boekingsverzoeken moeten worden afgehandeld.

Creëer Agentdefinitie: `agent_definition = await client.agents.create_agent(...)`: Maakt een agentdefinitie met het opgegeven model, de naam en de instructies.

Creëer AzureAI Agent: `agent = AzureAIAgent(...)`: Maakt een AzureAI-agent met behulp van de client, agentdefinitie en de gedefinieerde plugin.

Creëer Thread: `thread: AzureAIAgentThread | None = None`: Maakt een thread voor de agent. Het is niet vereist om eerst een thread te maken - als de waarde `None` wordt opgegeven, wordt een nieuwe thread gemaakt tijdens de eerste aanroep en als onderdeel van de respons teruggegeven.

Gebruikersinvoer: `user_inputs = ["..."]`: Definieert een lijst met gebruikersinvoer die de agent moet verwerken.

In het finally-blok, verwijder de thread en agent om middelen op te ruimen.


# Authenticatie

De `DefaultAzureCredential`-klasse maakt deel uit van de Azure SDK voor Python. Het biedt een standaardmanier om te authenticeren bij Azure-services. Het probeert te authenticeren met meerdere methoden in een specifieke volgorde, zoals omgevingsvariabelen, managed identity en Azure CLI-referenties.

Asynchrone Operaties: De aio-module geeft aan dat de DefaultAzureCredential-klasse asynchrone operaties ondersteunt. Dit betekent dat je het kunt gebruiken met asyncio om niet-blokkerende authenticatieverzoeken uit te voeren.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
